## Initial Exploration of API Wrapper

In [ ]:
#installing API Wrapper
# pip install MLB-StatsAPI

In [ ]:
#Importing API Wrapper
import statsapi

In [ ]:
#Checking homerun leaders for 2022
leaders = statsapi.league_leaders('homeRuns', season = 2022, limit = 5)
leaders = leaders.split('\n')
leaders

In [ ]:
#Looking at stats for a player - hitting and fielding specific
statsapi.player_stat_data(personId = 547180, group="[hitting,fielding]", type = 'season')

In [ ]:
#Looking at pitching stats for a player from a fetched player id year by year
statsapi.player_stat_data(personId = 592858, group="pitching", type = 'yearByYear')

In [ ]:
#Getting player info - will use for player IDs
statsapi.lookup_player('Willson Contreras')

In [ ]:
#Getting player ID for specific player
player_info = statsapi.lookup_player('Willson Contreras')
player_id = player_info[0]['id']
print(f'Player id is: {player_id}')

In [ ]:
#Looking at stats for a player from a fetched player id
statsapi.player_stat_data(personId = player_id, group="hitting", type = 'season')

In [ ]:
#Current team info
import datetime
statsapi.lookup_team(112, season = datetime.datetime.now().year)

In [ ]:
#Looking at season dates
statsapi.get("seasons", {"sportId": 1, "season": datetime.datetime.now().year})

In [ ]:
#Get past season data for a specific player
statsapi.get("people", {"personIds": 592450, "hydrate": "stats(group=[hitting],type=[byDateRange],\
    startDate=2021-04-07,endDate=2021-10-05,season=2021)"})

In [ ]:
#Get past season data for a specific player - hitting and pitching
statsapi.get("people", {"personIds": 543294, "hydrate": "stats(group=[hitting,pitching],type=[byDateRange],\
    startDate=2021-04-07,endDate=2021-05-21,season=2021)"})

In [ ]:
#Get hitting data for multiple players
statsapi.get("people", {"personIds": '660271,575929', "hydrate": "stats(group=[hitting],type=[byDateRange],\
    startDate=2021-04-07,endDate=2021-05-21,season=2021)"})

## Looking more into data and getting specifics

In [ ]:
#Get past season stats specific data for a player
player_info = statsapi.get("people", {"personIds": 592450, "hydrate": "stats(group=[hitting],type=[byDateRange],\
    startDate=2021-04-07,endDate=2021-05-21,season=2021)"})
player_info_stats = player_info['people'][0]['stats'][0]['splits'][0]['stat']
player_info_stats

In [ ]:
#Create function to get player stats
#Inputs are the id, start date and end date
def player_stats(id,start_date, end_date, season):
    player_info = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    stat_dict = {}
    try:
        player_hitting_stats = player_info['people'][0]['stats'][0]['splits'][0]['stat']
        stat_dict['hitting'] = player_hitting_stats
    except: pass
    try: 
        player_fielding_stats = player_info['people'][0]['stats'][1]['splits'][0]['stat']
        stat_dict['fielding'] = player_fielding_stats
    except: pass
    try:
        player_pitching_stats = player_info['people'][0]['stats'][2]['splits'][0]['stat']
        stat_dict['pitching'] = player_pitching_stats
    except:pass
    return stat_dict

In [ ]:
#Testing function for 2020
player_stats(592450,'2020-07-07','2020-10-22', 2020)

## Looking into getting a teams' roster

In [ ]:
#Getting Team Roster for a season
team_roster = statsapi.get("team_roster", {"teamId": 112, "hydrate": "stats(group=[hitting],type=[byDateRange],\
    startDate=2022-04-07,endDate=2022-10-05,season=2022)"})
player_info = team_roster['roster']
player_info

In [ ]:
#Looping through roster to get all player names for team 112
for i in range(len(player_info)):
    print(player_info[i]['person']['fullName'])

In [ ]:
#Creating list of all player names for team 112
[player_info[i]['person']['fullName'] for i in range(len(player_info))]

In [ ]:
#Creating function to get team roster for a team on a given year
#Input team id and the season year
def roster_names(team_id, season):
    team_roster = statsapi.get("team_roster", {"teamId": team_id, 'season': season,"hydrate": f"stats(group=[hitting],type=[byDateRange],season={season})"})
    player_info = team_roster['roster']
    #Creating list of all player names for a team 
    roster_list = [player_info[i]['person']['fullName'] for i in range(len(player_info))]
    return roster_list

In [ ]:
#Testing roster function
roster_names(112, 2021)

In [ ]:
#Creating function to get team roster for a team as dictionary
#Input team id, start date and end date and the season year
def roster(team_id, season):
    roster_dict = {}
    team_roster = statsapi.get("team_roster", {"teamId": team_id, 'season': season})
    player_info = team_roster['roster']
    #Creating list of all player names for a team 
    for i in range(len(player_info)):
        roster_dict[player_info[i]['person']['id']] = player_info[i]['person']['fullName']
    return roster_dict

In [ ]:
#Testing roster function
roster(112,2021)

In [ ]:
#Creating function to get team roster, including their position, for a team as dictionary
#Input team id, start date and end date and the season year
def roster_pos(team_id, season):
    roster_dict = {}
    team_roster = statsapi.get("team_roster", {"teamId": team_id, 'season': season})
    player_info = team_roster['roster']
    #Creating list of all player names for a team 
    for i in range(len(player_info)):
        roster_dict[player_info[i]['person']['id']] = [player_info[i]['person']['fullName'],player_info[i]['position']['type']]
    return roster_dict

In [ ]:
#Testing roster_pos function
roster_pos(108,2022)

In [ ]:
#Testing where the name and ID are located in the roster api call.
statsapi.get("team_roster", {"teamId": 112, 'season': 2021})['roster'][0]['person']

## Getting all the teams in the MLB

In [ ]:
#Testing API get for all the teams
all_teams = statsapi.get("teams", {"teamId": all,"hydrate": f"stats(group=[hitting],type=[byDateRange], season=2022)"})
all_teams

In [ ]:
#Printing all the NL teams as dict
for i in range(len(all_teams['teams'])):
    try:
        if all_teams['teams'][i]['league']['id']==104:
            print({all_teams['teams'][i]['id']:all_teams['teams'][i]['name']})
    except: pass

In [ ]:
#Printing all the AL teams as dict
for i in range(len(all_teams['teams'])):
    try:
        if all_teams['teams'][i]['league']['id']==103:
            print(all_teams['teams'][i]['id'])
    except: pass

In [ ]:
#Returns list of team ids in AL or NL (103 or 104)
def team_ids_choose(id):
    team_dict = {}
    for i in range(len(all_teams['teams'])):
        try:
            if all_teams['teams'][i]['league']['id']==id:
                team_dict[all_teams['teams'][i]['id']] = all_teams['teams'][i]['name']
        except: pass
    return team_dict

In [ ]:
#Testing function for AL
team_ids_choose(103)

In [ ]:
#Returns list of team ids in AL and NL
def team_id_list():
    team_list = []
    for i in range(len(all_teams['teams'])):
        try:
            if (all_teams['teams'][i]['league']['id']==103) or (all_teams['teams'][i]['league']['id']==104):
                team_list.append(all_teams['teams'][i]['id'])
        except: pass
    return team_list

In [ ]:
#Testing function
team_id_list()

In [ ]:
#Returns list of team ids in AL and NL
def team_id_dict():
    team_dict = {}
    for i in range(len(all_teams['teams'])):
        try:
            if (all_teams['teams'][i]['league']['id']==103) or (all_teams['teams'][i]['league']['id']==104):
                team_dict[all_teams['teams'][i]['id']] = all_teams['teams'][i]['name']
        except: pass
    return team_dict

In [ ]:
#Testing team dictionary for MLB teams
team_id_dict()

## Getting a teams' stats - Unfinished

In [ ]:
#Creating team stats function
#Currently able to pull the team roster, need to update to pull the stats for each player.
def team_stats(team):
    for i in team_id_dict():
        if team_id_dict()[i] == team:
            team_roster = roster(i,2021)
    return team_roster

In [ ]:
team_stats('Chicago Cubs')

In [ ]:
#Looking up pitcher who is a DH
statsapi.lookup_player('shohei ohtani')

In [ ]:
#Getting the type of position played - may use this for pitchers who are hitters (Called 'Two-Way Player' by the API)
#using an if statement could work when fetching team data, as it would only have to iterate through 26 or so players per team.
statsapi.get("people", {"personIds": 660271, "hydrate": "stats(group=[hitting],type=[byDateRange],\
    startDate=2021-04-07,endDate=2021-05-21,season=2021)"})['people'][0]['primaryPosition']['type']

## Fetch stats for the Cubs roster for 2021

In [ ]:
#Getting roster for cubs for 2020
cubs_roster = roster_pos(112,2021)
cubs_roster

In [ ]:
#Getting stats for 2021 roster
#Need the exception, as some players on the roster may have no stats at this time...
for i in cubs_roster:
    try: print(player_stats(i,'2021-04-07','2021-10-05', 2021))
    except: pass

## Looking into Winning and Losing teams for matchups

In [ ]:
#Returns matchups in given period for two teams, with outcomes.
#Could be used to show who won/lost the game for my training data.
statsapi.schedule(start_date='07/01/2018',end_date='07/31/2018',team=143,opponent=121)

In [ ]:
#Finding winning and losing team for all matchups in a month - printing winner of first game
winning_team = statsapi.schedule(start_date='07/01/2018',end_date='07/31/2018',team=143,opponent=121)[0]['winning_team']
losing_team = statsapi.schedule(start_date='07/01/2018',end_date='07/31/2018',team=143,opponent=121)[0]['losing_team']
winning_team

In [ ]:
#Printing the winner of all 4 games in this time period
for i in range(len(statsapi.schedule(start_date='07/01/2018',end_date='07/31/2018',team=143,opponent=121))):
    winning_team = statsapi.schedule(start_date='07/01/2018',end_date='07/31/2018',team=143,opponent=121)[i]['winning_team']
    print(winning_team)

In [ ]:
#Printing the winner of all games in the 2021 season - use try, as some games will be postponed.
for i in range(len(statsapi.schedule(start_date='2021-04-07',end_date='2021-10-05',team=143,opponent=121))):
    try:
        winning_team = statsapi.schedule(start_date='2021-04-07',end_date='2021-10-05',team=143,opponent=121)[i]['winning_team']
    except: print('Game Postponed')
    print(winning_team)

In [81]:
#Function to return a list of dicts of outcomes for two teams seasonal matches (or period of time)
def season_match_outcomes(team_1, team_2, season_start, season_end):
    dict_list = []
    for i in range(len(statsapi.schedule(start_date=season_start, end_date=season_end, team=team_1, opponent=team_2))):
        match_dict = {}
        try:
            match_dict['game_id'] = statsapi.schedule(start_date=season_start, end_date=season_end, team=team_1, opponent=team_2)[i]['game_id']
            match_dict['win'] = statsapi.schedule(start_date=season_start, end_date=season_end, team=team_1, opponent=team_2)[i]['winning_team']
            match_dict['loss'] = statsapi.schedule(start_date=season_start, end_date=season_end, team=team_1, opponent=team_2)[i]['losing_team']
            dict_list.append(match_dict)
        except: pass
    return dict_list

In [ ]:
#Looking at all match outcomes for Phillies and Mets
season_match_outcomes(121,143,'2021-04-07','2021-10-05')

## Getting game specific data

In [ ]:
#look more into this - returns a play by play of game data
#statsapi.get("game", {"gamePk": 530769, "hydrate": "stats(group=[hitting],type=[bySeason])"})